In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

распакуем архивы с тренировочными и тестовыми данными

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r')
zip_ref.extractall('/kaggle/temp/dogs-vs-cats/')
zip_ref = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip', 'r')
zip_ref.extractall('/kaggle/temp/dogs-vs-cats/')
zip_ref.close()

In [ ]:
!ls /kaggle/temp/dogs-vs-cats/

создадим дериктории для тренировочных данных и перенесем их в новые дериктории

In [ ]:
!mkdir /kaggle/temp/dogs-vs-cats/train/dog/
!mkdir /kaggle/temp/dogs-vs-cats/train/cat/

In [ ]:
import shutil
for file in os.listdir('/kaggle/temp/dogs-vs-cats/train/'):
    if 'dog' in str(file) and 'jpg' in str(file):
        shutil.move('/kaggle/temp/dogs-vs-cats/train/'+str(file),'/kaggle/temp/dogs-vs-cats/train/dog/'+str(file))
    if 'cat' in str(file) and 'jpg' in str(file):
        shutil.move('/kaggle/temp/dogs-vs-cats/train/'+str(file),'/kaggle/temp/dogs-vs-cats/train/cat/'+str(file))

In [ ]:
!ls /kaggle/temp/dogs-vs-cats/train

создадим новые дериктории для валидационных данных и также перенесем их

In [ ]:
!mkdir /kaggle/temp/dogs-vs-cats/validate/
!mkdir /kaggle/temp/dogs-vs-cats/validate/dog/
!mkdir /kaggle/temp/dogs-vs-cats/validate/cat/

In [ ]:
!ls /kaggle/temp/dogs-vs-cats/

In [ ]:
for file in os.listdir('/kaggle/temp/dogs-vs-cats/train/dog/')[:1250]:
    shutil.move('/kaggle/temp/dogs-vs-cats/train/dog/'+str(file),'/kaggle/temp/dogs-vs-cats/validate/dog/'+str(file))
for file in os.listdir('/kaggle/temp/dogs-vs-cats/train/cat/')[:1250]:
    shutil.move('/kaggle/temp/dogs-vs-cats/train/cat/'+str(file),'/kaggle/temp/dogs-vs-cats/validate/cat/'+str(file))


создадим новые дериктории для тестовых данных и перенесем их в новые дериктории 

In [ ]:
!mkdir /kaggle/temp/dogs-vs-cats/test/
!mkdir /kaggle/temp/dogs-vs-cats/test/dog
!mkdir /kaggle/temp/dogs-vs-cats/test/cat

In [ ]:
for file in os.listdir('/kaggle/temp/dogs-vs-cats/train/dog/')[:1250]:
    shutil.move('/kaggle/temp/dogs-vs-cats/train/dog/'+str(file),'/kaggle/temp/dogs-vs-cats/test/dog/'+str(file))
for file in os.listdir('/kaggle/temp/dogs-vs-cats/train/cat/')[:1250]:
    shutil.move('/kaggle/temp/dogs-vs-cats/train/cat/'+str(file),'/kaggle/temp/dogs-vs-cats/test/cat/'+str(file))


In [ ]:
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/train/dog/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/train/cat/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/validate/dog/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/validate/cat/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/test/dog/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/test/cat/')))

преобразуем данные в удобный вид для тренировки нейронной сети

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = '/kaggle/temp/dogs-vs-cats/train/'
VALIDATION_DIR ='/kaggle/temp/dogs-vs-cats/validate/'
TEST_DIR ='/kaggle/temp/dogs-vs-cats/test/'

train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size = 32,
    shuffle=True,
    target_size=(32,32),
    class_mode='binary'
)


validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size = 32,
    target_size=(32,32),
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    batch_size = 32,
    target_size=(32,32),
    class_mode='binary'
)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32,32,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    shuffle=True,
    verbose=1)

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(acc_values) + 1)
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
f,ax=plt.subplots(1,2,figsize=(18,8))
ax[0].plot(epochs, acc_values, 'b', label='Training acc')
ax[0].plot(epochs, val_acc_values, 'r', label='Validation acc')
ax[0].set_title("метрика accuracy для тренировочной и валидационной выборки")
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend()
ax[0].grid()
ax[1].plot(epochs, loss_values, 'b', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'r', label='Validation loss')
ax[1].set_title("функции ошибок тренировочной и валидационной выборки")
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend()
ax[1].grid()
plt.show()

In [ ]:
scores = model.evaluate(test_generator, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

In [ ]:
!mkdir /kaggle/temp/dogs-vs-cats/test1/pred

In [ ]:
for file in os.listdir('/kaggle/temp/dogs-vs-cats/test1'):
    if 'jpg' in str(file):
        shutil.move('/kaggle/temp/dogs-vs-cats/test1/'+str(file),'/kaggle/temp/dogs-vs-cats/test1/pred/'+str(file))
   

создадим датасет с помощью данных полученных от обученной нейронной сети

In [ ]:
PRED_DIR ='/kaggle/temp/dogs-vs-cats/test1/'

pred_datagen = ImageDataGenerator(rescale=1/255)
pred_generator = pred_datagen.flow_from_directory(
    PRED_DIR,
    batch_size = 32,
    shuffle=False,
    target_size=(32,32),
    class_mode=None
)

In [ ]:
prediction = model.predict(pred_generator)

In [ ]:
prediction

In [ ]:
prediction_result = [np.around(k) for k in prediction]

In [ ]:
results=pd.DataFrame({"id":range(1, len(prediction_result)+1),"label":prediction_result})
results

In [ ]:
results['label'] = results.label.astype("int32")

вычислим процентное содержание кошек и собак в датасете

In [ ]:
results.label.value_counts()

In [ ]:
max_persetage = results.label.value_counts()[0] + results.label.value_counts()[1]
dog_persantage = results.label.value_counts()[1] / max_persetage
cat_persantage = results.label.value_counts()[0] / max_persetage

In [ ]:
print("процент кошек в датасете составляет {}%, а процент собак {}%".format(cat_persantage*100, dog_persantage*100))

In [ ]:
objects = ('Cats', 'Dogs')
y_pos = np.arange(len(objects))
performance = [cat_persantage*100,dog_persantage*100]

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('%')
plt.title('процентное содержание кошек и собак')

plt.show()